<a href="https://colab.research.google.com/github/pranoyk/training-QA-model/blob/main/training_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import json
with open(r"train.json", "r") as read_file:
  train = json.load(read_file)

In [3]:
train

[{'context': "module icons { \n yang-version 1.1; \n namespace 'http://anutanetworks.com/icons'; \n prefix ico; \n\n organization \n 'Anuta Networks'; \n \n revision 2021-06-30 { \n description \n 'Report Infra Icons'; \n } \n\n container icons { \n list category { \n key 'name'; \n leaf name { \n type string; \n } \n leaf description { \n type string; \n } \n list icon { \n key 'name'; \n leaf name { \n type string; \n } \n leaf description { \n type string; \n } \n leaf url { \n type string; \n } \n } \n } \n } \n } \n. Based on the above YANG model return me RESTCONF-API method and request body for the questions asked, [Method: GET; API: /restconf/data/icons:icons/category]",
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Retrieve all the available icons?',
    'answers': [{'text': 'Method: GET; API: /restconf/data/icons:icons/category',
      'answer_start': 580}]},
   {'id': '00002',
    'is_impossible': False,
    'question': 'Give me all the icons?',
    

In [5]:
import json
with open(r"test.json", "r") as read_file:
  test = json.load(read_file)

In [6]:
test

[{'context': "module alert-group { \n namespace 'http://example.com/alert-group'; \n prefix alert-group; \n  \ncontainer alert-groups { \n container config { \n leaf-list name { \n type string; \n } \n } \n list group-config { \n key 'tenant'; \n leaf tenant { \n type string; \n } \n leaf-list name { \n type string; \n } \n } \n } \n }. Based on the above YANG model return me RESTCONF-API method and request body for the questions asked, [Method: GET; API: /restconf/data/alert-group:alert-groups]",
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Retrieve all the alert groups?',
    'answers': [{'text': 'Method: GET; API: /restconf/data/alert-group:alert-groups',
      'answer_start': 432}]},
   {'id': '00002',
    'is_impossible': False,
    'question': 'Can you give me all the alert groups?',
    'answers': [{'text': 'Method: GET; API: /restconf/data/alert-group:alert-groups',
      'answer_start': 432}]}]}]

In [7]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [35]:
import torch
torch.cuda.empty_cache()

In [8]:
model_type = "bert"
model_name = "bert-base-uncased"

In [ ]:
!pip install wandb

In [9]:
args = QuestionAnsweringArgs()


In [26]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 20,
    "evaluate_during_training_steps": 1000,
    # "wandb_project": "Question Answer Application",
    # "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size": 3,
    "train_batch_size": 128,
    "eval_batch_size": 64,
}

In [27]:
!rm -rf outputs

In [30]:
model = QuestionAnsweringModel(
    model_type, model_name, args=train_args
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [31]:
model.train_model(train, eval_data=test)

add example index and unique id: 100%|██████████| 5/5 [00:00<00:00, 40175.33it/s]


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 0 of 20:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


KeyError: ignored

In [16]:
results, text = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
results

{'correct': 0, 'similar': 2, 'incorrect': 0, 'eval_loss': -1.087890625}

In [18]:
text

{'correct_text': {},
 'similar_text': {'00001': {'truth': 'Method: GET; API: /restconf/data/alert-group:alert-groups',
   'predicted': '',
   'question': 'Retrieve all the alert groups?'},
  '00002': {'truth': 'Method: GET; API: /restconf/data/alert-group:alert-groups',
   'predicted': '',
   'question': 'Can you give me all the alert groups?'}},
 'incorrect_text': {}}

In [22]:
import json
with open(r"predictions.json", "r") as read_file:
  to_predict = json.load(read_file)

In [23]:
to_predict

[{'context': "module feature-toggle { \n namespace 'http://example.com/feature-toggle'; \n prefix ft; \n \n revision 2020-05-14 { \n description \n 'Initial revision'; \n } \n \n container feature-toggles { \n list feature-toggle { \n 0description \n 'The feature toggles enables a state information to decide if a feature/functionality should be enabled or disabled.'; \n key 'name'; \n leaf name { \n type string; \n description \n 'This is a predefined string value, usually populated by seed data only. This predefined key can be used to identify the features'; \n mandatory true; \n } \n leaf enable { \n type boolean; \n description \n 'This value tells whether to activate the feature'; \n } \n leaf description { \n type string; \n description \n 'Brief description about the feature toggle'; \n } \n } \n } \n }. Based on the above YANG model return me RESTCONF-API method and request body for the questions asked, [Method: GET; API: /restconf/data/feature-toggle:feature-toggles/feature-tog

In [29]:
final_answer, probablities = model.predict(to_predict)
print(final_answer)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 8322.03it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ["description 'Initial revision'; } container feature-toggles { list feature-toggle", "pace 'http://example.com/feature-toggle'; prefix ft", "namespace 'http://example.com/feature-toggle'; prefix ft; revision 2020-05-14 { description 'Initial revision'; } container feature-toggles { list feature-toggle"]}]
